In [14]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from mpl_toolkits.basemap import Basemap, addcyclic
%matplotlib qt
pd.set_option('mode.chained_assignment',None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

In [7]:
lat_0 = 52
lat_1 = 46
lon_0 = 6
lon_1 = 12
file_era_anom_p10 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p10.nc'.format(lon_0,lon_1,lat_0,lat_1)
file_era_anom_p90 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p90.nc'.format(lon_0,lon_1,lat_0,lat_1)
file_hw_pers = 'persistent_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
file_hw_short = 'short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
file_cw_pers = 'persistent_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
file_cw_short = 'short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
file_independent_hw_short = 'independent_short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
file_independent_cw_short = 'independent_short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
era_anom_p90 = xr.open_dataset(path+file_era_anom_p90).squeeze()
era_anom_p10 = xr.open_dataset(path+file_era_anom_p10).squeeze()

Heatwaves

In [15]:

pers_hw = np.load(path + file_hw_pers)
short_hw = np.load(path + file_hw_short)
independent_short_hw = np.load(path+file_independent_hw_short)
columns = ['start_date','end_date','length','mean_anomaly','max_anomaly']

file_namez = [file_hw_pers,file_hw_short]
variablez = [pers_hw,short_hw]
for j,variable in enumerate(variablez):
    index = np.arange(len(variable))
    df = pd.DataFrame(index=index,columns=columns)
    for i in range(len(variable)):
        ds = era_anom_p90.sel(time=slice(variable[i,0],variable[i,1]-pd.Timedelta(days=1)))
        ds_mean = ds.t.mean().values
        ds_max = ds.t.max().values
        length = len(ds.t)
        start = variable[i,0]-pd.Timedelta(days=0)
        end = variable[i,1]-pd.Timedelta(days=1)
        df['start_date'][i]=start
        df['end_date'][i]=end
        df['length'][i]=length
        df['mean_anomaly'][i]=ds_mean
        df['max_anomaly'][i]=ds_max
        if j==1:
            df['independent']=independent_short_hw
        df.to_csv(path+file_namez[j][:-4]+'.csv')
    


Coldwaves

In [16]:

pers_cw = np.load(path + file_cw_pers)
short_cw = np.load(path + file_cw_short)
independent_short_cw = np.load(path+file_independent_cw_short)

columns = ['start_date','end_date','length','mean_anomaly','min_anomaly']
index = np.arange(len(pers_hw))
file_namez = [file_cw_pers,file_cw_short]
variablez = [pers_cw,short_cw]
for j,variable in enumerate(variablez):
    index = np.arange(len(variable))
    df = pd.DataFrame(index=index,columns=columns)
    for i in range(len(variable)):
        ds = era_anom_p10.sel(time=slice(variable[i,0],variable[i,1]-pd.Timedelta(days=1)))
        ds_mean = ds.t.mean().values
        ds_min = ds.t.min().values
        length = len(ds.t)
        start = variable[i,0]-pd.Timedelta(days=0)
        end = variable[i,1]-pd.Timedelta(days=1)
        df['start_date'][i]=start
        df['end_date'][i]=end
        df['length'][i]=length
        df['mean_anomaly'][i]=ds_mean
        df['min_anomaly'][i]=ds_min
        if j==1:
            df.loc[:,'independent']=independent_short_cw
        df.to_csv(path+file_namez[j][:-4]+'.csv')